# Comparative analyses of machine learning-based models
This juptyer notebook goes through the compiling, training, testing and visualisation of machine learning models used for bacterial taxonomy classification using the curated 16S rRNA sequence datasets.

## Loading modules and training variables

In [98]:
# LOADING PACKAGES
import numpy as np
import pandas as pd
import sys
from varname import argname
import time
import os
import glob
import keras
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import f1_score, matthews_corrcoef, accuracy_score

from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten, Masking, Dot, Add, BatchNormalization
from keras.layers import MaxPooling1D, AveragePooling1D, Conv1D, Reshape
from keras.layers import TimeDistributed, LSTM, Bidirectional

import wandb
from wandb.keras import WandbCallback

LR, BATCH_SIZE, EPOCHS, = 0.001, 128, 60
MAX_LEN, MAX_LEN_V, INPUT_SHAPE_RNN, INPUT_SHAPE_RNN_V, = 2000, 500, (2000, 4), (500, 4)    # cut-off length
INPUT_SHAPE_3_MER, INPUT_SHAPE_5_MER, INPUT_SHAPE_7_MER = (5**3, 1), (5**5, 1), (5**7, 1)   # n**k

print('Packages loaded!')

Packages loaded!


## Loading and preprocessing the data

In [23]:
# LOADING THE NON-AUGMENTED AND AUGMENTED DATASETS
train_na = pd.read_csv('df_train_0.csv')
val_na = pd.read_csv('df_val_0.csv')
test_na = pd.read_csv('df_test_0.csv')
# ------------------------------------------------
train_a = pd.read_csv('df_train_1.csv')
val_a = pd.read_csv('df_val_1.csv')
# ------------------------------------------------
print('Datasets loaded!')

Datasets loaded!


### Generating k-mers
For use in the Convolutional Neural Networks (CNN), the sequences are processed into a frequency table of k-mers.

In [25]:
from itertools import product
# define all possible k-mers
alphabet = "AGTCN"
tri_mers = [''.join(chars) for chars in product(*(3*(alphabet,)))]
fiv_mers = [''.join(chars) for chars in product(*(5*(alphabet,)))]
sev_mers = [''.join(chars) for chars in product(*(7*(alphabet,)))]

def one_hot_k(sequence, kmers):
    k = len(kmers[0])
    # define a counter dictionary
    kmer_dict = dict.fromkeys(kmers, 0)

    # standardize the sequence
    # by replacing U with T and all ambiguous bases with N
    sequence = sequence.replace('U', 'T').replace('Y', 'N').replace('R', 'N').replace('W', 'N').replace('S', 'N').replace('K', 'N').replace('M', 'N').replace('D', 'N').replace('V', 'N').replace('H', 'N').replace('B', 'N').replace('X', 'N').replace('-', 'N')
    # count every k-mer in the sequence
    for i in range(0, len(sequence) - k+1):
        kmer_dict[sequence[i:i+k]] += 1

    # k-mer frequency array from the dictionary values
    k_array = np.array(list(kmer_dict.values()))
    # normalizing the array by dividing every value with the highest count value
    k_array = k_array / np.amax(k_array)
    return k_array

### One-hot-encoding the sequences
For use in the various Recurrent Neural Networks (RNN), the nucleotide sequences are processed into a one-hot-encoded format.

In [7]:
# Dictionary without consideration of mutation rate
one_hot_dict0 = {
    'A': [1.,0.,0.,0.], 'G':[0.,1.,0.,0.], 'T':[0.,0.,1.,0.], 'U':[0.,0.,1.,0.], 'C':[0.,0.,0.,1.], 
    'Y':[0.,0.,0.5,0.5], 'R':[0.5,0.5,0.,0.], 'W':[0.5,0.,0.5,0.], 'S':[0.,0.5,0.,0.5], 'K':[0.,0.5,0.5,0.], 'M':[0.5,0.,0.,0.5], 
    'D':[0.33,0.33,0.33,0.], 'V':[0.33,0.33,0.,0.33], 'H':[0.33,0.,0.33,0.33], 'B':[0.,0.33,0.33,0.33], 
    'X':[0.25,0.25,0.25,0.25], 'N':[0.25,0.25,0.25,0.25], '-':[0.,0.,0.,0.]
    }
# Dictionary with consideration of mutation rate (transition >> transversion)
one_hot_dict1 = {
    'A': [1.,0.,-0.5,-0.5], 'G':[0.,1.,-0.5,-0.5], 'T':[-0.5,-0.5,1.,0.], 'U':[-0.5,-0.5,1.,0.], 'C':[-0.5,-0.5,0.,1.], 
    'Y':[-0.5,-0.5,0.5,0.5], 'R':[0.5,0.5,-0.5,-0.5], 'W':[0.5,-0.5,0.5,-0.5], 'S':[-0.5,0.5,-0.5,0.5], 'K':[-0.5,0.5,0.5,-0.5], 'M':[0.5,-0.5,-0.5,0.5], 
    'D':[0.33,0.33,0.33,-1.], 'V':[0.33,0.33,-1.,0.33], 'H':[0.33,-1.,.33,0.33], 'B':[-1.,0.33,0.33,0.33], 
    'X':[0.,0.,0.,0.], 'N':[0.,0.,0.,0.], '-':[0.,0.,0.,0.]
    }

def one_hot_seq(sequence, one_hot_dict, max_len):
    # padding the sequences to a fixed length
	sequence += '-'*(max_len - len(sequence))
    # generating list of one-hot-lists using the dictionary
	onehot_encoded = [one_hot_dict[nucleotide] for nucleotide in sequence]
    # returning the list of lists as a numpy array
	return np.array(onehot_encoded)

### One-hot-encoding the labels
For use in the deep learning models, the labels are processed into a one-hot-encoding format. To achieve this, every unique label is first encoded to a numerical value.

In [27]:
def get_taxon_dict(df, taxon):
    # listing all unique taxon labels
    taxon_list = list(df[taxon].unique())

    # generating a dictionary to associate every unique taxon to a number
    taxon_dict = dict(zip(taxon_list, range(0, len(taxon_list))))
    # and the reversed dictionary as a lookup table
    taxon_dict_lookup = {v: k for k, v in taxon_dict.items()}

    return taxon_dict, taxon_dict_lookup

## Generating the input sequences and labels
The encoding methods for nucleotide sequences (x) defined earlier are now applied. The labels (y) are one-hot-encoded using the to_categorical function in keras_utils, thereby, converting the data into the correct format for feeding it to the deep learning models.

Every array is saved to reduce memory requirements down the road.

### Encoding the sequences into k-mer counts and one-hot-encoded sequences

In [28]:
# ENCODING SEQUENCES for the CNN models in 2 processing variations
# FOR CNN  |  with 3-mer encoding
x_train_CNN_na3 = np.array(
    train_na['Sequence'].apply(lambda x: one_hot_k(x, tri_mers)).tolist())
np.save('arrays/CNN/x_train_CNN_na3.npy', x_train_CNN_na3)

# x_train_CNN_a3 = np.array(
#   train_a['Sequence'].apply(lambda x: one_hot_k(x, tri_mers)).tolist())
# np.save('arrays/CNN/x_train_CNN_a3.npy', x_train_CNN_a3)

x_test_CNN_na3 = np.array(
    test_na['Sequence'].apply(lambda x: one_hot_k(x, tri_mers)).tolist())
np.save('arrays/CNN/x_test_CNN_na3.npy', x_test_CNN_na3)

dataval_CNN_na3 = np.array(
    val_na['Sequence'].apply(lambda x: one_hot_k(x, tri_mers)).tolist())
np.save('arrays/CNN/dataval_CNN_na3.npy', dataval_CNN_na3)

# dataval_CNN_a3 = np.array(
#   val_a['Sequence'].apply(lambda x: one_hot_k(x, tri_mers)).tolist())
# np.save('arrays/CNN/dataval_CNN_a3.npy', dataval_CNN_a3)
print('3-mer encoding complete')
# -----------------------------------------------------------------------------------------------------
# FOR CNN  |  with 5-mer encoding
x_train_CNN_na5 = np.array(
    train_na['Sequence'].apply(lambda x: one_hot_k(x, fiv_mers)).tolist())
np.save('arrays/CNN/x_train_CNN_na5.npy', x_train_CNN_na5)

# x_train_CNN_a5 = np.array(
#   train_a['Sequence'].apply(lambda x: one_hot_k(x, fiv_mers)).tolist())
# np.save('arrays/CNN/x_train_CNN_a5.npy', x_train_CNN_a5)

x_test_CNN_na5 = np.array(
    test_na['Sequence'].apply(lambda x: one_hot_k(x, fiv_mers)).tolist())
np.save('arrays/CNN/x_test_CNN_na5.npy', x_test_CNN_na5)

dataval_CNN_na5 = np.array(
    val_na['Sequence'].apply(lambda x: one_hot_k(x, fiv_mers)).tolist())
np.save('arrays/CNN/dataval_CNN_na5.npy', dataval_CNN_na5)

# dataval_CNN_a5 = np.array(
#   val_a['Sequence'].apply(lambda x: one_hot_k(x, fiv_mers)).tolist())
# np.save('arrays/CNN/dataval_CNN_a5.npy', dataval_CNN_a5)
print('5-mer encoding complete')
# -----------------------------------------------------------------------------------------------------
# FOR CNN  |  with 7-mer encoding
x_train_CNN_na7 = np.array(
    train_na['Sequence'].apply(lambda x: one_hot_k(x, sev_mers)).tolist())
np.save('arrays/CNN/x_train_CNN_na7.npy', x_train_CNN_na7)

x_train_CNN_a7 = np.array(
    train_a['Sequence'].apply(lambda x: one_hot_k(x, sev_mers)).tolist())
np.save('arrays/CNN/x_train_CNN_a7.npy', x_train_CNN_a7)

x_test_CNN_na7 = np.array(
    test_na['Sequence'].apply(lambda x: one_hot_k(x, sev_mers)).tolist())
np.save('arrays/CNN/x_test_CNN_na7.npy', x_test_CNN_na7)

dataval_CNN_na7 = np.array(
    val_na['Sequence'].apply(lambda x: one_hot_k(x, sev_mers)).tolist())
np.save('arrays/CNN/dataval_CNN_na7.npy', dataval_CNN_na7)

dataval_CNN_a7 = np.array(
    val_a['Sequence'].apply(lambda x: one_hot_k(x, sev_mers)).tolist())
np.save('arrays/CNN/dataval_CNN_a7.npy', dataval_CNN_a7)
print('7-mer encoding complete')
# -----------------------------------------------------------------------------------------------------
# FOR CNN  |  with 7-mer encoding on V-region selected sequences
x_train_CNN_v7 = np.array(
    train_na['V_Sequence'].apply(lambda x: one_hot_k(x, sev_mers)).tolist())
np.save('arrays/CNN/x_train_CNN_v7.npy', x_train_CNN_v7)

# x_train_CNN_va7 = np.array(
#     train_a['V_Sequence'].apply(lambda x: one_hot_k(x, sev_mers)).tolist())
# np.save('arrays/CNN/x_train_CNN_va7.npy', x_train_CNN_va7)

x_test_CNN_v7 = np.array(
    test_na['V_Sequence'].apply(lambda x: one_hot_k(x, sev_mers)).tolist())
np.save('arrays/CNN/x_test_CNN_v7.npy', x_test_CNN_v7)

dataval_CNN_v7 = np.array(
    val_na['V_Sequence'].apply(lambda x: one_hot_k(x, sev_mers)).tolist())
np.save('arrays/CNN/dataval_CNN_v7.npy', dataval_CNN_v7)

# dataval_CNN_va7 = np.array(
#     val_a['V_Sequence'].apply(lambda x: one_hot_k(x, sev_mers)).tolist())
# np.save('arrays/CNN/dataval_CNN_va7.npy', dataval_CNN_va7)
print('7-mer encoding complete')
# -----------------------------------------------------------------------------------------------------
print('CNN sequences complete')

3-mer complete
6-mer complete
CNN sequences complete


In [11]:
# ENCODING SEQUENCES for the RNN models in 2 processing variations
# FOR RNN  |  with regular one-hot-encoding
x_train_RNN_na0 = np.array(
    train_na['Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict0, max_len = MAX_LEN)).tolist())
np.save('arrays/RNN/x_train_RNN_na0.npy', x_train_RNN_na0)

# x_train_RNN_a0 = np.array(
#   train_a['Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict0, max_len = MAX_LEN)).tolist())
# np.save('arrays/RNN/x_train_RNN_a0.npy', x_train_RNN_a0)

x_test_RNN_na0 = np.array(
    test_na['Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict0, max_len = MAX_LEN)).tolist())
np.save('arrays/RNN/x_test_RNN_na0.npy', x_test_RNN_na0)

dataval_RNN_na0 = np.array(
    val_na['Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict0, max_len = MAX_LEN)).tolist())
np.save('arrays/RNN/dataval_RNN_na0.npy', dataval_RNN_na0)

# dataval_RNN_a0 = np.array(
#   val_a['Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict0, max_len = MAX_LEN)).tolist())
# np.save('arrays/RNN/dataval_RNN_a0.npy', dataval_RNN_a0)
print('Regular one-hot-encoding complete')
# -----------------------------------------------------------------------------------------------------
# FOR RNN  |  with matation rate adjusted one-hot-encoding
x_train_RNN_na1 = np.array(
    train_na['Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict1, max_len = MAX_LEN)).tolist())
np.save('arrays/RNN/x_train_RNN_na1.npy', x_train_RNN_na1)

x_train_RNN_a1 = np.array(
    train_a['Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict1, max_len = MAX_LEN)).tolist())
np.save('arrays/RNN/x_train_RNN_a1.npy', x_train_RNN_a1)

x_test_RNN_na1 = np.array(
    test_na['Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict1, max_len = MAX_LEN)).tolist())
np.save('arrays/RNN/x_test_RNN_na1.npy', x_test_RNN_na1)

dataval_RNN_na1 = np.array(
    val_na['Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict1, max_len = MAX_LEN)).tolist())
np.save('arrays/RNN/dataval_RNN_na1.npy', dataval_RNN_na1)

dataval_RNN_a1 = np.array(
    val_a['Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict1, max_len = MAX_LEN)).tolist())
np.save('arrays/RNN/dataval_RNN_a1.npy', dataval_RNN_a1)
print('Mutations rate adjusted one-hot-encoding complete')
# -----------------------------------------------------------------------------------------------------
# FOR RNN  |  with matation rate adjusted one-hot-encoding on the V-region selected sequences
x_train_RNN_v1 = np.array(
    train_na['V_Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict1, max_len = MAX_LEN_V)).tolist())
np.save('arrays/RNN/x_train_RNN_na1.npy', x_train_RNN_na1)

# x_train_RNN_va1 = np.array(
#   train_a['V_Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict1, max_len = MAX_LEN_V)).tolist())
# np.save('arrays/RNN/x_train_RNN_a1.npy', x_train_RNN_a1)

x_test_RNN_v1 = np.array(
    test_na['V_Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict1, max_len = MAX_LEN_V)).tolist())
np.save('arrays/RNN/x_test_RNN_na1.npy', x_test_RNN_na1)

dataval_RNN_v1 = np.array(
    val_na['V_Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict1, max_len = MAX_LEN_V)).tolist())
np.save('arrays/RNN/dataval_RNN_na1.npy', dataval_RNN_na1)

# dataval_RNN_va1 = np.array(
#   val_a['V_Sequence'].apply(lambda x: one_hot_seq(x, one_hot_dict1, max_len = MAX_LEN_V)).tolist())
# np.save('arrays/RNN/dataval_RNN_a1.npy', dataval_RNN_a1)
print('Mutations rate adjusted one-hot-encoding V-region complete')
# -----------------------------------------------------------------------------------------------------
print('RNN sequences complete')

Regular one-hot-encoding complete
Mutations rate adjusted one-hot-encoding complete
RNN sequences complete


### Encoding the labels into a one-hot-encoded format at Family level

In [10]:
taxon = 'Family'
taxon_dict = get_taxon_dict(test_na, taxon)[0]
# -----------------------------------------------------------------------------
# Associate every entry's label in the df to a number using the dictionary 
#   & one-hot encode the numerical labels
#   & save the result as a numpy array
y_train_fam_na = to_categorical(
    y = train_na[taxon].map(taxon_dict).astype(np.float32))
np.save('arrays/family/y_train_fam_na.npy', y_train_fam_na)

# y_train_fam_a = to_categorical(
#   y = train_a[taxon].map(taxon_dict).astype(np.float32))
# np.save('arrays/family/y_train_fam_a.npy', y_train_fam_a)

y_test_fam_na = to_categorical(
    y = test_na[taxon].map(taxon_dict).astype(np.float32))
np.save('arrays/family/y_test_fam_na.npy', y_test_fam_na)

labelsval_fam_na = to_categorical(
    y = val_na[taxon].map(taxon_dict).astype(np.float32))
np.save('arrays/family/labelsval_fam_na.npy', labelsval_fam_na)

# labelsval_fam_a = to_categorical(
#   y = val_a[taxon].map(taxon_dict).astype(np.float32))
# np.save('arrays/family/labelsval_fam_a.npy', labelsval_fam_a)
print('Family label arrays generated')
# -----------------------------------------------------------------------------
fam_count = train_na[taxon].nunique()
print(f'The number of unique family labels: {fam_count}')

Labels complete
Family train/test/val arrays generated
The number of unique family labels: 349


### Encoding the labels into a one-hot-encoded format at Genus level

In [13]:
taxon = 'Genus'
taxon_dict = get_taxon_dict(test_na, taxon)[0]
# -----------------------------------------------------------------------------
y_train_gen_na = to_categorical(
    y = train_na[taxon].map(taxon_dict).astype(np.float32))
np.save('arrays/genus/y_train_gen_na.npy', y_train_gen_na)

y_train_gen_a = to_categorical(
    y = train_a[taxon].map(taxon_dict).astype(np.float32))
np.save('arrays/genus/y_train_gen_a.npy', y_train_gen_a)

y_test_gen_na = to_categorical(
    y = test_na[taxon].map(taxon_dict).astype(np.float32))
np.save('arrays/genus/y_test_gen_na.npy', y_test_gen_na)

labelsval_gen_na = to_categorical(
    y = val_na[taxon].map(taxon_dict).astype(np.float32))
np.save('arrays/genus/labelsval_gen_na.npy', labelsval_gen_na)

labelsval_gen_a = to_categorical(
    y = val_a[taxon].map(taxon_dict).astype(np.float32))
np.save('arrays/genus/labelsval_gen_a.npy', labelsval_gen_a)
print('Genus label arrays generated')
# -----------------------------------------------------------------------------
gen_count = train_na[taxon].nunique()
print(f'The number of unique genus labels: {gen_count}')

Genus label arrays generated
The number of unique genus labels: 954


### Encoding the labels into a one-hot-encoded format at Species level

In [12]:
taxon = 'Species'
taxon_dict = get_taxon_dict(test_na, taxon)[0]
# -----------------------------------------------------------------------------
y_train_spe_na = to_categorical(
    y = train_na[taxon].map(taxon_dict).astype(np.float32))
np.save('arrays/species/y_train_spe_na.npy', y_train_spe_na)

# y_train_spe_a = to_categorical(
#   y = train_a[taxon].map(taxon_dict).astype(np.float32))
# np.save('arrays/species/y_train_spe_a.npy', y_train_spe_a)

y_test_spe_na = to_categorical(
    y = test_na[taxon].map(taxon_dict).astype(np.float32))
np.save('arrays/species/y_test_spe_na.npy', y_test_spe_na)

labelsval_spe_na = to_categorical(
    y = val_na[taxon].map(taxon_dict).astype(np.float32))
np.save('arrays/species/labelsval_spe_na.npy', labelsval_spe_na)

# labelsval_spe_a = to_categorical(
#   y = val_a[taxon].map(taxon_dict).astype(np.float32))
# np.save('arrays/species/labelsval_spe_a.npy', labelsval_spe_a)
print('Species label arrays generated')
# -----------------------------------------------------------------------------
spe_count = train_na[taxon].nunique()
print(f'The number of unique species labels: {spe_count}')

Species train/test/val arrays generated
The number of unique species labels: 1569


## Loading the generated training/validation/testing arrays
When seperating the processing and training/evaluation scripts, due to memory limits, the saved arrays can be loaded using this script.

In [ ]:
# AMOUNT OF UNIQUE LABELS AT EACH TAXON LEVEL
fam_count, gen_count, spe_count = 349, 954, 1569

# LOADING ENCODED SEQUENCES for the CNN models in 2 processing variations
# FOR CNN  |  with 3-mer
x_train_CNN_na3 = np.load('arrays/CNN/x_train_CNN_na3.npy')
# x_train_CNN_a3 = np.load('arrays/CNN/x_train_CNN_a3.npy')
x_test_CNN_na3 = np.load('arrays/CNN/x_test_CNN_na3.npy')
dataval_CNN_na3 = np.load('arrays/CNN/dataval_CNN_na3.npy')
# dataval_CNN_a3 = np.load('arrays/CNN/dataval_CNN_a3.npy')
print('3-mer encoded sequences LOADED')
# -----------------------------------------------------------------------------
# FOR CNN  |  with 5-mer
x_train_CNN_na5 = np.load('arrays/CNN/x_train_CNN_na5.npy')
# x_train_CNN_a5 = np.load('arrays/CNN/x_train_CNN_a5.npy')
x_test_CNN_na5 = np.load('arrays/CNN/x_test_CNN_na5.npy')
dataval_CNN_na5 = np.load('arrays/CNN/dataval_CNN_na5.npy')
# dataval_CNN_a5 = np.load('arrays/CNN/dataval_CNN_a5.npy')
print('5-mer encoded sequences LOADED')
# -----------------------------------------------------------------------------
# FOR CNN  |  with 7-mer
x_train_CNN_na7 = np.load('arrays/CNN/x_train_CNN_na7.npy')
x_train_CNN_a7 = np.load('arrays/CNN/x_train_CNN_a7.npy')
x_test_CNN_na7 = np.load('arrays/CNN/x_test_CNN_na7.npy')
dataval_CNN_na7 = np.load('arrays/CNN/dataval_CNN_na7.npy')
dataval_CNN_a7 = np.load('arrays/CNN/dataval_CNN_a7.npy')
print('7-mer encoded sequences LOADED')
# -----------------------------------------------------------------------------
# FOR CNN  |  with 7-mer on V-region selected data
x_train_CNN_v7 = np.load('arrays/CNN/x_train_CNN_v7.npy')
# x_train_CNN_va7 = np.load('arrays/CNN/x_train_CNN_va7.npy')
x_test_CNN_v7 = np.load('arrays/CNN/x_test_CNN_v7.npy')
dataval_CNN_v7 = np.load('arrays/CNN/dataval_CNN_v7.npy')
# dataval_CNN_va7 = np.load('arrays/CNN/dataval_CNN_va7.npy')
print('7-mer encoded V-region sequences LOADED')
# -----------------------------------------------------------------------------
print('CNN sequences LOADED')

###############################################################################

# LOADING ENCODED SEQUENCES for the RNN models in 2 processing variations
# FOR RNN  |  with regular one-hot-encoding
x_train_RNN_na0 = np.load('arrays/RNN/x_train_RNN_na0.npy')
# x_train_RNN_a0 = np.load('arrays/RNN/x_train_RNN_a0.npy')
x_test_RNN_na0 = np.load('arrays/RNN/x_test_RNN_na0.npy')
dataval_RNN_na0 = np.load('arrays/RNN/dataval_RNN_na0.npy')
# dataval_RNN_a0 = np.load('arrays/RNN/dataval_RNN_a0.npy')
print('Regular one-hot-encoded sequences LOADED')
# -----------------------------------------------------------------------------
# FOR RNN  |  with matation rate adjusted one-hot-encoding
x_train_RNN_na1 = np.load('arrays/RNN/x_train_RNN_na1.npy')
x_train_RNN_a1 = np.load('arrays/RNN/x_train_RNN_a1.npy')
x_test_RNN_na1 = np.load('arrays/RNN/x_test_RNN_na1.npy')
dataval_RNN_na1 = np.load('arrays/RNN/dataval_RNN_na1.npy')
dataval_RNN_a1 = np.load('arrays/RNN/dataval_RNN_a1.npy')
print('Mutation rate adjusted one-hot-encoded sequences LOADED')
# -----------------------------------------------------------------------------
# FOR RNN  |  with matation rate adjusted encoding on V-region selected data
x_train_RNN_v1 = np.load('arrays/RNN/x_train_RNN_v1.npy')
# x_train_RNN_va1 = np.load('arrays/RNN/x_train_RNN_va1.npy')
x_test_RNN_v1 = np.load('arrays/RNN/x_test_RNN_v1.npy')
dataval_RNN_v1 = np.load('arrays/RNN/dataval_RNN_v1.npy')
# dataval_RNN_va1 = np.load('arrays/RNN/dataval_RNN_va1.npy')
print('Mutation rate adjusted one-hot-encoded V-region sequences LOADED')
# -----------------------------------------------------------------------------
print('RNN sequences LOADED')

###############################################################################

# LOADING one-hot encoded labels at each taxon level
# -----------------------------------------------------------------------------
# LABELS AT FAMILY LEVEL
y_train_fam_na = np.load('arrays/family/y_train_fam_na.npy')
# y_train_fam_a = np.load('arrays/family/y_train_fam_a.npy')
y_test_fam_na = np.load('arrays/family/y_test_fam_na.npy')
labelsval_fam_na = np.load('arrays/family/labelsval_fam_na.npy')
# labelsval_fam_a = np.load('arrays/family/labelsval_fam_a.npy')
print('Family label arrays LOADED')
# -----------------------------------------------------------------------------
# LABELS AT GENUS LEVEL
y_train_gen_na = np.load('arrays/genus/y_train_gen_na.npy')
y_train_gen_a = np.load('arrays/genus/y_train_gen_a.npy')
y_test_gen_na = np.load('arrays/genus/y_test_gen_na.npy')
labelsval_gen_na = np.load('arrays/genus/labelsval_gen_na.npy')
labelsval_gen_a = np.load('arrays/genus/labelsval_gen_a.npy')
print('Genus label arrays LOADED')
# -----------------------------------------------------------------------------
# LABELS AT SPECIES LEVEL
y_train_spe_na = np.load('arrays/species/y_train_spe_na.npy')
# y_train_spe_a = np.load('arrays/species/y_train_spe_a.npy')
y_test_spe_na = np.load('arrays/species/y_test_spe_na.npy')
labelsval_spe_na = np.load('arrays/species/labelsval_spe_na.npy')
# labelsval_spe_a = np.load('arrays/species/labelsval_spe_a.npy')
print('Species label arrays LOADED')

## Setting up the network architectures
What follows are a set of functions for creating the deep learning models.

### 1. Convolutional Neural Network (CNN)

In [107]:
# CNN
def make_CNNmodel(input_shape, out_len, name = 'CNN'):
    CNNmodel = keras.Sequential(
        [
            Reshape(target_shape = input_shape, input_shape = input_shape[:-1]),
            Conv1D(4, 15, input_shape = input_shape),
            Activation('relu'),
            MaxPooling1D(pool_size = 2),

            Conv1D(8, 10),
            Activation('relu'),
            MaxPooling1D(pool_size = 2),

            Conv1D(12, 5),
            Activation('relu'),
            MaxPooling1D(pool_size = 2),
            Dropout(0.2),

            Flatten(),
            Dense(256),
            Activation('relu'),
            Dropout(0.4),

            Dense(out_len, activation='softmax')
        ], 
        name = name
    )
    return CNNmodel

### 2. Bidirectional Long-Short Term Memory Neural Network (BiLSTM)

In [30]:
# BiLSTM
def make_BiLSTMmodel(input_shape, out_len, name = 'BiLSTM'):
    BiLSTMmodel = keras.Sequential(
        [
            Masking(mask_value = 0., input_shape = input_shape),
            
            Bidirectional(LSTM(128, return_sequences = True), merge_mode = 'sum'),
            Dropout(0.5),

            AveragePooling1D(4),
            Bidirectional(LSTM(128), merge_mode = 'sum'),
            Dropout(0.5),

            Dense((out_len), activation = 'softmax'),
        ],
        name = name
    )
    return BiLSTMmodel

### 3. Convolutional BiLSTM Neural Network (ConvBiLSTM)

In [103]:
# ConvBiLSTM
def make_ConvBiLSTMmodel(input_shape, out_len, name = 'ConvBiLSTM'):
    ConvBiLSTMmodel = keras.Sequential(
        [
            Masking(mask_value = 0., input_shape = input_shape),
                        
            Conv1D(128, 3, padding = 'same'),
            AveragePooling1D(),

            Conv1D(128, 3, padding = 'same'),
            AveragePooling1D(),

            Conv1D(128, 3, padding = 'same', use_bias = True),
            AveragePooling1D(),
            Dropout(0.4),
            
            Bidirectional(LSTM(128, activation = 'tanh'), merge_mode = 'sum'),
            Dropout(0.2),
            
            Dense(128, activation = 'relu'),
            Dropout(0.2),
            Dense(out_len, activation = 'softmax')
        ], 
        name = name
    )
    return ConvBiLSTMmodel

### 4. Attention-based ConvBiLSTM (Read2Pheno)

In [32]:
# Read2Pheno
## Conv & Res net layers
CONV_NET_nr, RES_NET_nr, NET_filters, NET_window = 2, 1, 64, 2
## extra Dropout layer (after Res block)
DROP_r, POOL_s = 0.2, 2
## BiLSTM layer
LSTM_nodes, MERGE_m = 128, 'sum'
## attention Layers
ATT_layers, ATT_nodes = 1, 128
## fully connected layers
FC_layers, FC_nodes, FC_drop = 1, 128, 0.3

#####################################################################################################
# BLOCK FUNCTIONS
def conv_net_block(X, n_cnn_filters = 256, cnn_window = 9, block_name = 'convblock'):
    '''
    convolutional block with a 1D convolutional layer, a batch norm layer followed by a relu activation.
    parameters:
        n_cnn_filters: number of output channels
        cnn_window: window size of the 1D convolutional layer
    '''
    X = Conv1D(n_cnn_filters, cnn_window, strides = 1, padding = 'same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    return X

def res_net_block(X, n_cnn_filters = 256, cnn_window = 9, block_name = 'resblock'):
    '''
    residual net block accomplished by a few convolutional blocks.
    parameters:
        n_cnn_filters: number of output channels
        cnn_window: window size of the 1D convolutional layer
    '''
    X_identity = X
    # cnn0
    X = Conv1D(n_cnn_filters, cnn_window, strides = 1, padding = 'same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    # cnn1
    X = Conv1D(n_cnn_filters, cnn_window, strides = 1, padding = 'same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    # cnn2
    X = Conv1D(n_cnn_filters, cnn_window, strides = 1, padding = 'same')(X)
    X = BatchNormalization()(X)
    X = Add()([X, X_identity])
    X = Activation('relu')(X)
    return X

def attention_layer(H_lstm, n_layer, n_node, block_name = 'att'):
    '''
    feedforward attention layer accomplished by time distributed dense layers.
    parameters:
        n_layer: number of hidden layers
        n_node: number of hidden nodes
    '''
    H_emb = H_lstm
    for i in range(n_layer):
        H_lstm = TimeDistributed(Dense(n_node, activation = "tanh"))(H_lstm)
    M = TimeDistributed(Dense(1, activation = "linear"))(H_lstm)
    alpha = keras.layers.Softmax(axis = 1)(M)
    r_emb = Dot(axes = 1)([alpha, H_emb])
    r_emb = Flatten()(r_emb)
    return r_emb

def fully_connected(r_emb, n_layer, n_node, drop_out_rate = 0.5, block_name = 'fc'):
    '''
    fully_connected layer consists of a few dense layers.
    parameters:
        n_layer: number of hidden layers
        n_node: number of hidden nodes
        drop_out_rate: dropout rate to prevent the model from overfitting
    '''
    for i in range(n_layer):
        r_emb = Dense(n_node, activation = "relu")(r_emb)
    r_emb = Dropout(drop_out_rate)(r_emb) 
    return r_emb
    
#####################################################################################################
# MODEL COMPILING FUNCTION

def make_R2Pmodel(input_shape, out_len, name = 'Read2Pheno'):
    X = Input(shape = input_shape)
    X_mask = Masking(mask_value = 0.)(X)

    ## CONV Layers
    X_cnn = X_mask
    # conv_net
    for i in range(CONV_NET_nr):
        X_cnn = conv_net_block(
            X_cnn, 
            n_cnn_filters = NET_filters, 
            cnn_window = NET_window
            )
    # res_net
    for i in range(RES_NET_nr):
        X_cnn = res_net_block(
            X_cnn, 
            n_cnn_filters = NET_filters, 
            cnn_window = NET_window
            )

    ## Extra Pooling layer and Dropout
    X_pool = AveragePooling1D(pool_size = POOL_s)(X_cnn)
    X_drop = Dropout(DROP_r)(X_pool)

    ## RNN Layers
    H_lstm = Bidirectional(LSTM(LSTM_nodes, return_sequences = True), merge_mode = MERGE_m)(X_drop)
    H_lstm = Activation('tanh')(H_lstm)

    ## ATT Layers
    r_emb = attention_layer(
        H_lstm, 
        n_layer = ATT_layers, 
        n_node = ATT_nodes, 
        block_name = 'att'
        )    
    # Fully connected layers
    r_emb = fully_connected(
        r_emb, 
        n_layer = FC_layers, 
        n_node = FC_nodes, 
        drop_out_rate = FC_drop, 
        block_name = 'fc'
        )

    # Compile model
    out = Dense(out_len, activation = 'softmax', name = 'final_dense')(r_emb)
    R2Pmodel = Model(inputs = X, outputs = out, name = name)
    
    return R2Pmodel

## Creating the Deep Learning models
The models are generated based on input and output shapes.

### 1. CNN
The CNN models are created, tailored to the different input (k-mer) and output (taxon) shapes. The input shape is determined by the k-mer used (with associated matrix) and the output shape (and thus the number of nodes in the final dense layer) is determined by the amount of unique taxon labels.

In [33]:
# ----------------------------------------------------------
# for Family
CNN_fam_3 = make_CNNmodel(
    input_shape = INPUT_SHAPE_3_MER, 
    out_len = fam_count, 
    name = 'CNN_fam_3'
    )
CNN_fam_5 = make_CNNmodel(
    input_shape = INPUT_SHAPE_5_MER, 
    out_len = fam_count, 
    name = 'CNN_fam_5'
    )
CNN_fam_7 = make_CNNmodel(
    input_shape = INPUT_SHAPE_7_MER, 
    out_len = fam_count, 
    name = 'CNN_fam_7'
    )
# ----------------------------------------------------------
# for Genus
CNN_gen_3 = make_CNNmodel(
    input_shape = INPUT_SHAPE_3_MER, 
    out_len = gen_count, 
    name = 'CNN_gen_3'
    )
CNN_gen_5 = make_CNNmodel(
    input_shape = INPUT_SHAPE_5_MER, 
    out_len = gen_count, 
    name = 'CNN_gen_5'
    )
CNN_gen_7 = make_CNNmodel(
    input_shape = INPUT_SHAPE_7_MER, 
    out_len = gen_count, 
    name = 'CNN_gen_7'
    )
# ----------------------------------------------------------
# for Species
CNN_spe_3 = make_CNNmodel(
    input_shape = INPUT_SHAPE_3_MER, 
    out_len = spe_count, 
    name = 'CNN_spe_3'
    )
CNN_spe_5 = make_CNNmodel(
    input_shape = INPUT_SHAPE_5_MER, 
    out_len = spe_count, 
    name = 'CNN_spe_5'
    )
CNN_spe_7 = make_CNNmodel(
    input_shape = INPUT_SHAPE_7_MER, 
    out_len = spe_count, 
    name = 'CNN_spe_7'
    )

### 2. RNN
The RNN models are created, tailored to the different input (sequence length) and output (taxon) shapes. The input shape is determined by the length of the sequences used (V-region selected or not) and the output shape (and thus the number of nodes in the final dense layer) is determined by the amount of unique taxon labels.

In [34]:
# ----------------------------------------------------------
# for Family (output length = number of unique labels)
BiLSTM_fam = make_BiLSTMmodel(
    input_shape = INPUT_SHAPE_RNN,
    out_len = fam_count, 
    name = 'BiLSTM_fam'
    )
ConvBiLSTM_fam = make_ConvBiLSTMmodel(
    input_shape = INPUT_SHAPE_RNN,
    out_len = fam_count
    , name = 'ConvBiLSTM_fam'
    )
R2P_fam = make_R2Pmodel(
    input_shape = INPUT_SHAPE_RNN,
    out_len = fam_count, 
    name = 'R2P_fam'
    )
# ----------------------------------------------------------
# for Genus
BiLSTM_gen = make_BiLSTMmodel(
    input_shape = INPUT_SHAPE_RNN,
    out_len = gen_count, 
    name = 'BiLSTM_gen'
    )
ConvBiLSTM_gen = make_ConvBiLSTMmodel(
    input_shape = INPUT_SHAPE_RNN,
    out_len = gen_count, 
    name = 'ConvBiLSTM_gen'
    )
R2P_gen = make_R2Pmodel(
    input_shape = INPUT_SHAPE_RNN,
    out_len = gen_count, 
    name='R2P_gen'
    )
# ----------------------------------------------------------
# for Species
BiLSTM_spe = make_BiLSTMmodel(
    input_shape = INPUT_SHAPE_RNN,
    out_len = spe_count, 
    name = 'BiLSTM_spe'
    )
ConvBiLSTM_spe = make_ConvBiLSTMmodel(
    input_shape = INPUT_SHAPE_RNN,
    out_len = spe_count, 
    name = 'ConvBiLSTM_spe'
    )
R2P_spe = make_R2Pmodel(
    input_shape = INPUT_SHAPE_RNN,
    out_len = spe_count, 
    name='R2P_spe'
    )
# ----------------------------------------------------------
# ----------------------------------------------------------
# for Genus with V-region selected data
BiLSTM_gen_V = make_BiLSTMmodel(
    input_shape = INPUT_SHAPE_RNN_V,
    out_len = gen_count, 
    name = 'BiLSTM_gen_V'
    )
ConvBiLSTM_gen_V = make_ConvBiLSTMmodel(
    input_shape = INPUT_SHAPE_RNN_V,
    out_len = gen_count, 
    name = 'ConvBiLSTM_gen_V'
    )
R2P_gen_V = make_R2Pmodel(
    input_shape = INPUT_SHAPE_RNN_V,
    out_len = gen_count, 
    name='R2P_gen_V'
    )

## Compiling, Training and Evaluating the Deep Learning models

For each model:
- a Weights and Biases run is initiated
- the model is compiled and a summary is printed
- the model is trained and training-time is measured
- the model is evaluated by calculating the test-loss and -accuracy, the F1 score and the MCC score
- the training history and metrics are saved

In [35]:
def train_and_evaluate_model(
    model, 
    train_data, train_labels, 
    validation_data, validation_labels, 
    test_data, test_labels
    ):

    wandb.init(project = 'Final Training', entity = 'bachelorprojectgroup9', name = model.name)

    # LOADING MODEL
    print ('Loading {} model...'.format(model.name))
    model.compile(
        loss='categorical_crossentropy', 
        optimizer=Adam(learning_rate=LR), 
        metrics=['accuracy']
        )
    print(model.summary())

    # FITTING MODEL
    print ('Fitting {} model...'.format(model.name))
    start_time = time.time()
    history = model.fit(
        train_data, train_labels, 
        epochs = EPOCHS, batch_size=BATCH_SIZE, 
        validation_data = (validation_data, validation_labels), 
        callbacks = [WandbCallback()]
        )
    time_taken = round(time.time() - start_time)
    # history object is saved and can later be destinguished using the model/train_data names
    np.save('history/{}_{}.npy'.format(model.name, argname('train_data')), history.history)
    
    # EVALUATING MODEL
    print ('Evaluating {} model...'.format(model.name))
    test_labels_arg = np.argmax(test_labels, axis = 1)
    test_predictions = np.argmax(model.predict(test_data), axis = 1)
    loss, accuracy = model.evaluate(test_data, test_labels)
    # F1-score: harmonic mean of the precision and recall
    #   score from 0 to 1
    f1 = f1_score(y_true = test_labels_arg, y_pred = test_predictions, average = 'weighted')
    # Matthews correlation coefficient: coefficient of +1 represents a perfect prediction,
    #   0 an average random prediction and -1 an inverse prediction
    mcc = matthews_corrcoef(y_true = test_labels_arg, y_pred = test_predictions)

    score_dict = pd.DataFrame({
        'Model/run' : model.name, 
        'Data' : argname('train_data'), 
        'Training time' : time_taken, 
        'Test loss' : loss, 
        'Test accuracy' : accuracy, 
        'F1-score' : f1, 
        'MCC' : mcc}, 
        index=[0]
        )
    print(score_dict)
    # score metrics are saved and can later be destinguished using the model names
    score_dict.to_csv('scores/{}_evaluation.csv'.format(model.name), index = False)

    wandb.finish()
    return

Running the CNN models and saving the results

In [ ]:
# RUNNING CNN MODELS
# ----------------------------------------------------------
# ----------------------------------------------------------
# running the CNN model at genus level 
# with 3-, 5- and 7-mer encoding
# on the non-augmented data
train_and_evaluate_model(
    CNN_gen_3, 
    x_train_CNN_na3, y_train_gen_na, 
    dataval_CNN_na3, labelsval_gen_na, 
    x_test_CNN_na3, y_test_gen_na
    )
train_and_evaluate_model(
    CNN_gen_5, 
    x_train_CNN_na5, y_train_gen_na, 
    dataval_CNN_na5, labelsval_gen_na, 
    x_test_CNN_na5, y_test_gen_na
    )
train_and_evaluate_model(
    CNN_gen_7, 
    x_train_CNN_na7, y_train_gen_na, 
    dataval_CNN_na7, labelsval_gen_na, 
    x_test_CNN_na7, y_test_gen_na
    )
# ----------------------------------------------------------
# ----------------------------------------------------------
# running the CNN model at family and species level
# with 7-mer encoding
# on the non-augmented data
train_and_evaluate_model(
    CNN_fam_7, 
    x_train_CNN_na7, y_train_fam_na, 
    dataval_CNN_na7, labelsval_fam_na, 
    x_test_CNN_na7, y_test_fam_na
    )
train_and_evaluate_model(
    CNN_spe_7, 
    x_train_CNN_na7, y_train_spe_na, 
    dataval_CNN_na7, labelsval_spe_na, 
    x_test_CNN_na7, y_test_spe_na
    )
# ----------------------------------------------------------
# ----------------------------------------------------------
# running the CNN model at genus level 
# with 7-mer encoding
# on the augmented data
train_and_evaluate_model(
    CNN_gen_7, 
    x_train_CNN_a7, y_train_gen_a, 
    dataval_CNN_a7, labelsval_gen_a, 
    x_test_CNN_na7, y_test_gen_na
    )
# ----------------------------------------------------------
# ----------------------------------------------------------
# running the CNN model at genus level 
# with 7-mer encoding
# on the V-region selected data
train_and_evaluate_model(
    CNN_gen_7, 
    x_train_CNN_v7, y_train_gen_na, 
    dataval_CNN_v7, labelsval_gen_na, 
    x_test_CNN_v7, y_test_gen_na
    )

Running the RNN models and saving the results

In [ ]:
# RUNNING RNN MODELS
# ----------------------------------------------------------
# ----------------------------------------------------------
# running the RNN models at genus level 
# with regualar and mutation rate adjusted one-hot-encoding
# on the non-augmented data
train_and_evaluate_model(
    BiLSTM_gen, 
    x_train_RNN_na0, y_train_gen_na, 
    dataval_RNN_na0, labelsval_gen_na, 
    x_test_RNN_na0, y_test_gen_na
    )
train_and_evaluate_model(
    ConvBiLSTM_gen, 
    x_train_RNN_na0, y_train_gen_na, 
    dataval_RNN_na0, labelsval_gen_na, 
    x_test_RNN_na0, y_test_gen_na
    )
train_and_evaluate_model(
    R2P_gen, 
    x_train_RNN_na0, y_train_gen_na, 
    dataval_RNN_na0, labelsval_gen_na, 
    x_test_RNN_na0, y_test_gen_na
    )
# ----------------------------------------------------------
train_and_evaluate_model(
    BiLSTM_gen, 
    x_train_RNN_na1, y_train_gen_na, 
    dataval_RNN_na1, labelsval_gen_na, 
    x_test_RNN_na1, y_test_gen_na
    )
train_and_evaluate_model(
    ConvBiLSTM_gen, 
    x_train_RNN_na1, y_train_gen_na, 
    dataval_RNN_na1, labelsval_gen_na, 
    x_test_RNN_na1, y_test_gen_na
    )
train_and_evaluate_model(
    R2P_gen, 
    x_train_RNN_na1, y_train_gen_na, 
    dataval_RNN_na1, labelsval_gen_na, 
    x_test_RNN_na1, y_test_gen_na
    )
# ----------------------------------------------------------
# ----------------------------------------------------------
# running the RNN models at family and species level 
# with the mutation rate adjusted one-hot-encoding
# on the non-augmented data
train_and_evaluate_model(
    BiLSTM_fam, 
    x_train_RNN_na1, y_train_fam_na, 
    dataval_RNN_na1, labelsval_fam_na, 
    x_test_RNN_na1, y_test_fam_na
    )
train_and_evaluate_model(
    ConvBiLSTM_fam, 
    x_train_RNN_na1, y_train_fam_na, 
    dataval_RNN_na1, labelsval_fam_na, 
    x_test_RNN_na1, y_test_fam_na
    )
train_and_evaluate_model(
    R2P_fam, 
    x_train_RNN_na1, y_train_fam_na, 
    dataval_RNN_na1, labelsval_fam_na, 
    x_test_RNN_na1, y_test_fam_na
    )
# ----------------------------------------------------------
train_and_evaluate_model(
    BiLSTM_spe, 
    x_train_RNN_na1, y_train_spe_na, 
    dataval_RNN_na1, labelsval_spe_na, 
    x_test_RNN_na1, y_test_spe_na
    )
train_and_evaluate_model(
    ConvBiLSTM_spe, 
    x_train_RNN_na1, y_train_spe_na, 
    dataval_RNN_na1, labelsval_spe_na, 
    x_test_RNN_na1, y_test_spe_na
    )
train_and_evaluate_model(
    R2P_spe, 
    x_train_RNN_na1, y_train_spe_na, 
    dataval_RNN_na1, labelsval_spe_na, 
    x_test_RNN_na1, y_test_spe_na
    )
# ----------------------------------------------------------
# ----------------------------------------------------------
# running the RNN models at genus level 
# with the mutation rate adjusted one-hot-encoding 
# on the augmented data
train_and_evaluate_model(
    BiLSTM_gen, 
    x_train_RNN_a1, y_train_gen_a, 
    dataval_RNN_a1, labelsval_gen_a, 
    x_test_RNN_na1, y_test_gen_na
    )
train_and_evaluate_model(
    ConvBiLSTM_gen, 
    x_train_RNN_a1, y_train_gen_a, 
    dataval_RNN_a1, labelsval_gen_a, 
    x_test_RNN_na1, y_test_gen_na
    )
train_and_evaluate_model(
    R2P_gen, 
    x_train_RNN_a1, y_train_gen_a, 
    dataval_RNN_a1, labelsval_gen_a, 
    x_test_RNN_na1, y_test_gen_na
    )
# ----------------------------------------------------------
# ----------------------------------------------------------
# running the RNN models at genus level 
# with the mutation rate adjusted one-hot-encoding 
# on the V-region selected data
train_and_evaluate_model(
    BiLSTM_gen_V, 
    x_train_RNN_v1, y_train_gen_na, 
    dataval_RNN_v1, labelsval_gen_na, 
    x_test_RNN_v1, y_test_gen_na
    )
train_and_evaluate_model(
    ConvBiLSTM_gen_V, 
    x_train_RNN_v1, y_train_gen_na, 
    dataval_RNN_v1, labelsval_gen_na, 
    x_test_RNN_v1, y_test_gen_na
    )
train_and_evaluate_model(
    R2P_gen_V, 
    x_train_RNN_v1, y_train_gen_na, 
    dataval_RNN_v1, labelsval_gen_na, 
    x_test_RNN_v1, y_test_gen_na
    )

## Ribosomal Database Project (RDP)
Train and validation dataset are merged into one, this is becuase the RDP machine learning model does not require validation steps during training. The databases are converted to ready4train taxonomy and fasta files to be fed into the RDP Classifier.

When running cells, new files will be saved in "RDPfiles" directory at the same folder of this jupyter notebook is located. Directory name can be changed.

### 1. Setting variables and loading data
Training and classification is done with the whole taxon hierarchy, from kingdom to species, but a specific level can be chosen during evaluation.

In [5]:
global RDPfiles
import pandas as pd
# runs to do
#   - genus level with 3 data types (non-augmented, augmented, V-selected)
#   - family and species level with non-augmented

# variables
RDPfiles = "RDPfiles"
classifier_loc = "rdptools/classifier.jar"
confidence_score = 0.8
level = 'genus'             # used for evaluation
model_run = 'RDP_gen'       # used for naming the run (change taxon)
data = 'x_train_RDP_naX'    # used for naming the data (change na/n/v)
seq_col = -1                # used for selecting sequence (change -1/-2 for V_seq col)
is_v = -1                   # used for selecting taxa (change to -2 if there is a V_seq col)
# loading data
train_data = pd.read.csv('df_train_0.csv')  # change 1/0 for a/na data
val_data = pd.read_csv('df_val_0.csv')      # change 1/0 for a/na data
test_data = pd.read_csv('df_test_0.csv')

print('Variables set and datasets loaded!')

### 2. Processing functions
**Lineage2taxTrain** converts tab separated taxonomy text files into the ready4train_taxonomy.txt file. This text file contains the hierarchical taxonomy information in the following format: tax ID | taxon name | parent taxid | depth | rank.
- Tax ID is the index of the rank in the taxonomy file
- Taxon name is the name for the taxonomic rank
- Parent taxid is the tax ID of the rank above the current rank
- Depth is the depth of the rank. Depth 0 is always root. The kingdom rank has depth of 1
- Rank is the taxonomic ranks

**AddFullLineage** generates the ready4train_seqs.fasta file. It has structure similar to the sequence fasta file, but semicolon separated taxonomy is added next to sequence ID.


**RDPoutput2score** generates the accuracy, F1 score and MCC using output.txt and test_taxonomy.txt. Output.txt is generated when classifying test_sequences.txt with the trained model.

In [3]:
def lineage2taxTrain(raw_taxons):
    taxons_list = raw_taxons.strip().split('\n')
    header = taxons_list[0].split('\t')[1:] # headers = list of ranks
    hash = {} # taxon name-id map
    ranks = {} # column number-rank map
    lineages = [] # list of unique lineages

    with open("{}/ready4train_taxonomy.txt".format(RDPfiles), "w") as f:
        # initiate root rank taxon id map
        hash = {"Root":0}
        for i in range(len(header)):
            name = header[i]
            ranks[i] = name

        # root rank info
        root = ['0', 'Root', '-1', '0', 'rootrank']
        f.write("*".join(root) +  '\n')

        ID = 0
        for line in taxons_list[1:]:
            cols = line.strip().split('\t')[1:]
            # iterate each column
            for i in range(len(cols)):
                name = []
                for node in cols[:i + 1]:
                    node = node.strip()
                    if not node in ('-', ''):
                        name.append(node)

                pName = ";".join(name[:-1])
                if not name in lineages:
                    lineages.append(name)

                depth = len(name)
                name = ';'.join(name)
                if name in hash.keys():
                    # already seen this lineage
                    continue
                try:
                    rank = ranks[i]
                except KeyError:
                    print (cols)
                    sys.exit()

                if i == 0:
                    pName = 'Root'
                # parent taxid
                pID = hash[pName]
                ID += 1
                # add name-id to the map
                hash[name] = ID
                out = ['%s'%ID, name.split(';')[-1], '%s'%pID, '%s'%depth, rank]
                f.write("*".join(out) + '\n')
    f.close()

def addFullLineage(raw_taxons, raw_seqs):
    # lineage map
    hash = {}
    taxonomy_list = raw_taxons.strip().split('\n')

    for line in taxonomy_list[1:]:
        line = line.strip()
        cols = line.strip().split('\t')
        lineage = ['Root']

        for node in cols[1:]:
            node = node.strip()
            if not (node == '-' or node == ''):
                lineage.append(node)

        ID = cols[0]
        lineage = ';'.join(lineage).strip()
        hash[ID] = lineage

    sequence_list = raw_seqs.strip().split('\n')
    with open("{}/ready4train_seqs.fasta".format(RDPfiles), "w") as f:
        for line in sequence_list:
            line = line.strip()
            if line == '':
                continue
            if line[0] == '>':
                ID = line.strip().split()[0].replace('>', '')
                lineage = hash[ID]
                f.write('>' + ID + '\t' + lineage + '\n')
            else:
                f.write(line.strip() + '\n')
    f.close()

def RDPoutput2score(pred_file, true_file, level, cf):
    taxon_list = []
    ranks = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
    level = ranks.index(level)

    pred = pd.read_csv(pred_file, sep="\t", header=None)
    pred.drop(pred.columns[1:level+5+2*level],  axis = 'columns', inplace=True)
    pred.drop(pred.columns[4:], axis = 'columns', inplace=True)
    
    pred_dict = {}
    for index, row in pred.iterrows():
        row = row.tolist()
        if row[1] not in taxon_list:
            taxon_list += [row[1]]
        if float(row[3]) >= cf:
            pred_dict[row[0]] = row[1]

    true = pd.read_csv(true_file, sep="\t", header=None)
    true_dict = {}
    for index, row in true.iterrows():
        true_dict[row[0]] = row[level+1]
        if row[level+1] not in taxon_list:
            taxon_list += [row[level+1]]


    y_pred, y_true = [], []
    for i in pred_dict.keys():
        y_pred.append(taxon_list.index(pred_dict[i]))
        y_true.append(taxon_list.index(true_dict[i]))

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    mcc = matthews_corrcoef(y_true, y_pred)

    score_dict = pd.DataFrame({
        'Model/run' : model_run,    # 'RDP_gen'
        'Data' : data,              # 'x_train_RDP_naX'
        'Training time' : None, 
        'Test loss' : None, 
        'Test accuracy' : acc, 
        'F1-score' : f1, 
        'MCC' : mcc}, 
        index=[0]
        )
    print(score_dict)
    return score_dict

### 3. RDP Main
In this cell, a new directory is made to save output.

- Train and validation sets are merged into one training set.
- raw_seqs + raw_taxons -> ready4train files (input for training the RDP model)
- test dataset -> test_sequcnes.fasta + test_taxonomy.txt (used in evaluation of new trained models)

In [43]:
# main_RDP.py
os.system("mkdir {}".format(RDPfiles))

# merge train and validation dataframes into one.
train = pd.concat([train_data, val_data], ignore_index = True)
test = test_data

# convert train and test dataframe into tab separated taxonomy and sequence string
# taxnomy file is converted to a tab separated string
# sequence file is converted to fasta format with sequence ID and sequence
raw_seqs = ''
raw_taxons = 'SeqId Kingdom	Phylum	Class	Order	Family	Genus	Species' + '\n'
for index, row in train.iterrows():
    taxons = row.tolist()
    raw_seqs += '>' + taxons[0] + '\n' + taxons[seq_col] + '\n'
    raw_taxons += '\t'.join(taxons[:is_v]) + '\n'

# convert test dataframe into text and fasta files to be utilized by RDP
# taxnomy file is converted to a tab separated text file
# sequence file is converted to fasta format
with open("{}/test_sequences.fasta".format(RDPfiles), "w") as seq_f, open(
    "{}/test_taxonomy.txt".format(RDPfiles), "w") as tax_f:

    for index, row in test.iterrows():
        taxons = row.tolist()
        seq_f.write('>' + taxons[0] + '\n' + taxons[seq_col] + '\n')
        tax_f.write('\t'.join(taxons[:is_v]) + '\n')
        
    seq_f.close()
    tax_f.close()

# convert raw taxonomy and sequence files to ready4rdp trainable files
lineage2taxTrain(raw_taxons)
addFullLineage(raw_taxons, raw_seqs)

print("Data preprocessing for RDP completed")

Data preprocessing for RDP completed


Training is performed with the train() function. The model is saved in the training_files directory with four weight files. New file, rRNAClassifier.properties, are necessary for bridging these files and the RDP Classifier. Classification is performed with the classify() function. Option -o leads RDP Classifier to use the newly generated training models.

Using RDPoutput2score, the accuracy, F1 score and MCC are calculated. These values will be used to compare with the deep learning models. User can choose specific level of taxon to evaluate prediction.

In [6]:
# Training the RDP classifier
start_time = time.time()
os.system("java -Xmx10g -jar {} train -o {}/training_files -s {}/ready4train_seqs.fasta -t {}/ready4train_taxonomy.txt".format(
    classifier_loc, RDPfiles, RDPfiles, RDPfiles))

with open("{}/training_files/rRNAClassifier.properties".format(RDPfiles), "w") as f:
    f.write("bergeyTree=bergeyTrainingTree.xml\nprobabilityList=genus_wordConditionalProbList.txt\nprobabilityIndex=wordConditionalProbIndexArr.txt\nwordPrior=logWordPrior.txt\nclassifierVersion=RDP Naive Bayesian rRNA Classifier Version 2.5, May 2012 ")
    f.close()

time_taken = round(time.time() - start_time)
print("RDP training-time: {} seconds".format(time_taken))

# Testing the RDP classifier
os.system("java -Xmx10g -jar {} classify -t {}/training_files/rRNAClassifier.properties  -o {}/output.txt {}/test_sequences.fasta".format(
    classifier_loc, RDPfiles, RDPfiles, RDPfiles))

# Evaluating the RDP classifier and save the results
score_dict = RDPoutput2score("{}/output.txt".format(RDPfiles), 
"{}/test_taxonomy.txt".format(RDPfiles), level, confidence_score)

score_dict.at[0, 'Training time'] = time_taken
score_dict.to_csv('scores/RDP_evaluation', index = False)

RDP training-time: 85 seconds
accuracy: 0.9724941724941725
F1-score: 0.9726149488062451
MCC-score: 0.9723559927778965


## Combining and Visualizing the results
### Loading the saved results

In [9]:
# importing the required modules
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

First the csv_files containing the scores are loaded and the resulting dataframe is shown.

In [47]:
 # specifying the path to csv files
path = "scores"
# csv files in the path
files = glob.glob(path + "/*.csv")


# defining an empty DataFrame
col_names = ['Model/run', 'Data', 'Training time', 'Test loss', 'Test accuracy', 'F1-score', 'MCC']
score_df = pd.DataFrame(columns = col_names)
# defining an empty list to store content
content = []

for filename in files:
    # reading content of csv file
    df = pd.read_csv(filename, index_col = None)
    content.append(df)
  
# converting content to one data frame
score_df = pd.concat(content)
score_df.sample(5)

,Model/run,Data,Training time,Test loss,Test accuracy,F1-score,MCC
0,BiLSTM_gen,x_train_RNN_na0,2677,2.569284,0.434009,0.364383,0.429371
0,CNN_gen_6,x_train_CNN_na6,346,0.270210,0.924773,0.913211,0.924266
0,CNN_gen_3,x_train_CNN_na3,193,0.923764,0.759431,0.732084,0.757753
0,CNN_fam_6,x_train_CNN_na6,224,0.088498,0.980857,0.979135,0.980552
0,ConvBiLSTM_gen,x_train_RNN_na0,628,1.829659,0.580320,0.531201,0.577239
0,CNN_spe_6,x_train_CNN_na6,225,0.607089,0.831412,0.799931,0.830663
0,R2P_gen,x_train_RNN_na0,1443,1.420991,0.662935,0.610153,0.660651


In [48]:
df = score_df.copy()

# splitting the column values in seperate, more informative columns
df.insert(loc = 0, column = 'Encoding', 
value = df['Data'].apply(lambda x: x.split('_')[-1][-1]))
df.insert(loc = 0, column = 'Taxon', 
value = df['Model/run'].apply(lambda x: x.split('_')[1]))
df.insert(loc = 0, column = 'Model', 
value = df['Model/run'].apply(lambda x: x.split('_')[0]))

df['Data'] = score_df['Data'].apply(lambda x: x.split('_')[-1][:2])

# removing the columns that are no longer necessary
df.drop('Model/run', axis=1, inplace=True)
df.sample(5)

,Model,Taxon,Encoding,Data,Training time,Test loss,Test accuracy,F1-score,MCC
0,BiLSTM,gen,0,na,2677,2.569284,0.434009,0.364383,0.429371
0,CNN,gen,6,na,346,0.270210,0.924773,0.913211,0.924266
0,CNN,gen,3,na,193,0.923764,0.759431,0.732084,0.757753
0,CNN,fam,6,na,224,0.088498,0.980857,0.979135,0.980552
0,ConvBiLSTM,gen,0,na,628,1.829659,0.580320,0.531201,0.577239
0,CNN,spe,6,na,225,0.607089,0.831412,0.799931,0.830663
0,R2P,gen,0,na,1443,1.420991,0.662935,0.610153,0.660651


In [49]:
# ----------------------------------------------------------------------------------
# Functions for cleaning the dataframe
# to generate a nicer looking table
def clean_taxon_name(cell):
    if cell == 'spe':
        cell = 'Species'
    elif cell == 'gen':
        cell = 'Genus'
    else:
        cell = 'Family'
    return cell

def clean_data_name(cell):
    if cell == 'na':
        cell = 'non-augmented'
    elif cell[0] == 'a':
        cell = 'augmented'
    else:
        cell = 'V-region selected'
    return cell

def clean_encoding_name(cell):
    # for RNN models
    if cell == '0':
        cell = 'regular one-hot'
    elif cell == '1':
        cell = 'mutation rate adjusted one-hot'
    # for RDP
    elif cell == 'X':
        cell = '8-mer with parentID'
    # for CNN models
    else:
        cell = cell + '-mer'
    return cell

def clean_time(sec):
    sec = sec % (24 * 3600)
    hr = sec // 3600
    sec %= 3600
    min = sec // 60
    sec %= 60
    return "{}:{}:{}".format(hr, min, sec)
# ----------------------------------------------------------------------------------

# making a copy of the dataframe to clean
clean_df = df.copy().sort_values('Model')
# applying the cleaning functions
clean_df['Taxon'] = clean_df['Taxon'].apply(lambda x: clean_taxon_name(x))
clean_df['Data'] = clean_df['Data'].apply(lambda x: clean_data_name(x))
clean_df['Encoding'] = clean_df['Encoding'].apply(lambda x: clean_encoding_name(x))
clean_df['Training time'] = clean_df['Training time'].apply(lambda x: clean_time(x))
# rounding the float values
clean_df[['Test loss', 'Test accuracy', 'F1-score', 'MCC']] = clean_df[[
    'Test loss', 'Test accuracy', 'F1-score', 'MCC'
    ]].round(4)
# converting test accuracy to percentage
clean_df['Test accuracy'] = clean_df['Test accuracy'].apply(lambda x: x*100)
clean_df = clean_df.rename(columns={"Test accuracy":"Test accuracy (%)"})

clean_df = clean_df.sort_values('Encoding').reset_index().iloc[: , 1:]
clean_df.sample(5)

,Model,Taxon,Encoding,Data,Training time,Test loss,Test accuracy (%),F1-score,MCC
0,CNN,Genus,3-mer,non-augmented,0:3:13,0.9238,75.94,0.7321,0.7578
1,CNN,Genus,6-mer,non-augmented,0:5:46,0.2702,92.48,0.9132,0.9243
2,CNN,Family,6-mer,non-augmented,0:3:44,0.0885,98.09,0.9791,0.9806
3,CNN,Species,6-mer,non-augmented,0:3:45,0.6071,83.14,0.7999,0.8307
4,BiLSTM,Genus,regular one-hot,non-augmented,0:44:37,2.5693,43.40,0.3644,0.4294
5,ConvBiLSTM,Genus,regular one-hot,non-augmented,0:10:28,1.8297,58.03,0.5312,0.5772
6,R2P,Genus,regular one-hot,non-augmented,0:24:3,1.4210,66.29,0.6102,0.6607


Next the npy_files containing the training history are loaded. This generates a dictionary of history_dictionaries.

In [41]:
history_files = glob.glob("history/*.npy")

histories_dict = {}
for np_name in history_files:
    # the name is cut to a better looking format
    name = np_name.split('\\')[1][:-4]
    histories_dict[name] = np.load(np_name, allow_pickle=True).item()

### Plotting and Discussing the training histories
Using the Seaborn package, the function plot_hist() is defined that returns plots for the desired comparison.

Using the Pandas package, the function give_score() is defined that returns the scores of the desired comparison.

As RDP does not have a history object, it is only compared against the deep learning models with the score table.

In [45]:
sns.set_style("dark")
sns.set_theme()

# list used as keys for which plots to make
plots = ['loss', 'accuracy', 'val_loss', 'val_accuracy']
# list used as titels to give the subplots
titles = ['Training Loss', 'Training Accuracy', 'Validation Loss', 'Validation Accuracy']
# list used as labels for y-axis (no unit of measurement for loss)
y_labels = ['', 'percentage (%)', '', 'percentage (%)']

def plot_hist(hist_dict_lst, epochs = 60, suptitle = 'Comparison_title'):
    # Initialise x, y and legend value lists
    x_values = np.arange(epochs + 1, dtype=int)
    df_lst = [
        pd.DataFrame({'Epochs' : x_values}), 
        pd.DataFrame({'Epochs' : x_values}), 
        pd.DataFrame({'Epochs' : x_values}), 
        pd.DataFrame({'Epochs' : x_values})
        ]
    legend_names = []
    # iterating over the histories to be plotted (compared runs)
    for key in hist_dict_lst:
        hist_dict = histories_dict[key]
        # ----------------------------------------------------------------------------------
        # the names are augmented and minimized 
        # to show all necessary information in a nice looking way
        name_lst = key.split('_')

        # making encoding method more informative
        if name_lst[0] == 'CNN':
            name_lst[2] = name_lst[2] + '-mer encoding'
        else:
            if name_lst[-1][-1] == 0:
                name_lst[2] = 'regular one-hot-encoding'
            else:
                name_lst[2] = 'mutation rate adjusted one-hot-encoding'
        
        # making taxon-level more informative
        if name_lst[1] == 'spe':
            name_lst[1] = 'at Species-level'
        elif name_lst[1] == 'gen':
            name_lst[1] = 'at Genus-level'
        else:
            name_lst[1] = 'at Family-level'

        # making data name more informative
        if name_lst[-1][:2] == 'na':
            name_lst[-1] = 'with non-augmented data'
        elif name_lst[-1][0] == 'a':
            name_lst[-1] = 'with augmented data'
        elif name_lst[-1][:2] == 'va':
            name_lst[-1] = 'with augmented V-region selected data'
        else:
            name_lst[-1] = 'with non-augmented V-region selected data'

        name = ' '.join(name_lst[:3]) + ' ' + name_lst[-1]
        legend_names.append(name)
        # ----------------------------------------------------------------------------------
        # iterating over the plots to make
        for i, plt_key in enumerate(plots):
            plt_values = hist_dict[plt_key].copy()
            # adding value for epoch 0 for nicer plots (arbitrary)
            # for loss
            if i == 0 or i == 2:
                plt_values.insert(0, 8)
            # for accuracy
            if i == 1 or i == 3:
                plt_values.insert(0, 0)
                # converting accuracy to percentage
                plt_values = [plt_value*100 for plt_value in plt_values]
            # add value to corresponding dataframe
            df_lst[i][key] = plt_values
    # --------------------------------------------------------------------------------------
    # initialise the figure with 4 subplots (2 rows & 2 columns)
    fig, ax = plt.subplots(2, 2, figsize = (18, 12))
    # iterate over the subplots with the corresponding dataframe
    frame = 0
    for row in range(2):
        for col in range(2):
            # melt dataframe for Seaborn to accept it (differentiating run based on hue)
            dfm = df_lst[frame].melt('Epochs', var_name = 'Run', value_name = 'values')
            plt.rcParams["axes.titlesize"] = 16
            sns.lineplot(
                data = dfm, x = 'Epochs', y = 'values', hue = 'Run', 
                ax = ax[row, col], legend = False).set(
                    title = titles[frame], 
                    xlim = (0, epochs), ylim = (0, None), 
                    ylabel = y_labels[frame])
            frame += 1
    # --------------------------------------------------------------------------------------
    # adding a legend and title for the total figure
    fig.legend(
        labels = legend_names, 
        title = 'Compared runs (model / encoding / taxon / data)', 
        loc = 1)
    fig.suptitle(
        suptitle, 
        horizontalalignment = 'center', 
        fontsize = 22,
        fontweight = 'bold')

def give_score(model, taxon, encoding, data, df = clean_df):
    return df.loc[
        (df['Model'].isin(model)) & 
        (df['Data'].isin(data)) &
        (df['Encoding'].isin(taxon)) & 
        (df['Data'].isin(data))
        ]

#### Defining Comparisons
The desired comparisons are defined and lists containing the keys for looking up the history_dictionaries are generated.

In [43]:
# -----------------------------------------------------------------
# Lists of history objects for comparing each model
# with various encoding methods
histories_CNN_encoding_genus = [
    'CNN_gen_3_x_train_CNN_na3', 
    'CNN_gen_5_x_train_CNN_na5',
    'CNN_gen_7_x_train_CNN_na7'
    ]
histories_BiLSTM_encoding_genus = [
    'BiLSTM_gen_x_train_RNN_na0', 
    'BiLSTM_gen_x_train_RNN_na1'
    ]
histories_ConvBiLSTM_encoding_genus = [
    'ConvBiLSTM_gen_x_train_RNN_na0', 
    'ConvBiLSTM_gen_x_train_RNN_na1'
    ]
histories_R2P_encoding_genus = [
    'R2P_gen_x_train_RNN_na0', 
    'R2P_gen_x_train_RNN_na1'
    ]
# -----------------------------------------------------------------
# Lists of history objects for comparing each model
# at the three taxon levels
histories_CNN_taxa = [
    'CNN_fam_7_x_train_CNN_na7', 
    'CNN_gen_7_x_train_CNN_na7', 
    'CNN_spe_7_x_train_CNN_na7'
    ]
histories_BiLSTM_taxa = [
    'BiLSTM_fam_x_train_RNN_na1', 
    'BiLSTM_gen_x_train_RNN_na1', 
    'BiLSTM_spe_x_train_RNN_na1'
    ]
histories_ConvBiLSTM_taxa = [
    'ConvBiLSTM_fam_x_train_RNN_na1', 
    'ConvBiLSTM_gen_x_train_RNN_na1', 
    'ConvBiLSTM_spe_x_train_RNN_na1'
    ]
histories_R2P_taxa = [
    'R2P_fam_x_train_RNN_na1', 
    'R2P_gen_x_train_RNN_na1', 
    'R2P_spe_x_train_RNN_na1'
    ]
# -----------------------------------------------------------------
# Lists of history objects for comparing each model
# with regular/augmented/V-region selected data
histories_CNN_data = [
    'CNN_gen_7_x_train_CNN_na7', 
    'CNN_gen_7_x_train_CNN_a7',
    'CNN_gen_7_x_train_CNN_v7'
    ]
histories_BiLSTM_data = [
    'BiLSTM_gen_x_train_RNN_na1', 
    'BiLSTM_gen_x_train_RNN_a1',
    'BiLSTM_gen_x_train_RNN_v1'
    ]
histories_ConvBiLSTM_data = [
    'ConvBiLSTM_gen_x_train_RNN_na1', 
    'ConvBiLSTM_gen_x_train_RNN_a1',
    'ConvBiLSTM_gen_x_train_RNN_v1'
    ]
histories_R2P_data = [
    'R2P_gen_x_train_RNN_na1', 
    'R2P_gen_x_train_RNN_a1',
    'R2P_gen_x_train_RNN_v1'
    ]
# -----------------------------------------------------------------
# Lists of history objects for comparing the models with each other
#   at species-level on non-augmeted data
histories_models_spe = [
    'CNN_spe_7_x_train_CNN_na7', 
    'BiLSTM_spe_x_train_RNN_na1', 
    'ConvBiLSTM_spe_x_train_RNN_na1', 
    'R2P_spe_x_train_RNN_na1'
    ]
#   at genus-level on non-augmeted data
histories_models_gen = [
    'CNN_gen_7_x_train_CNN_na7', 
    'BiLSTM_gen_x_train_RNN_na1', 
    'ConvBiLSTM_gen_x_train_RNN_na1', 
    'R2P_gen_x_train_RNN_na1'
    ]
#   at family-level on non-augmeted data
histories_models_fam = [
    'CNN_fam_7_x_train_CNN_na7', 
    'BiLSTM_fam_x_train_RNN_na1', 
    'ConvBiLSTM_fam_x_train_RNN_na1', 
    'R2P_fam_x_train_RNN_na1'
    ]
#   at genus-level on augmeted data
histories_models_a = [
    'CNN_gen_7_x_train_CNN_a7', 
    'BiLSTM_gen_x_train_RNN_a1', 
    'ConvBiLSTM_gen_x_train_RNN_a1', 
    'R2P_gen_x_train_RNN_a1'
    ]
#   at genus-level on V-region selected data
histories_models_v = [
    'CNN_gen_7_x_train_CNN_v7', 
    'BiLSTM_gen_x_train_RNN_v1', 
    'ConvBiLSTM_gen_x_train_RNN_v1', 
    'R2P_gen_x_train_RNN_v1'
    ]

#### 1. Comparing the models' results with the various encoding methods
**Comparing the CNN model at Genus level with 3-, 5- and 7-mer encoded sequences**



In [ ]:
print(give_score(
    model = ['CNN'], 
    taxon = ['Genus'], 
    encoding = ['3-mer', '5-mer', '7-mer'], 
    data = ['non-augmented']
    ))

plot_hist(
    histories_CNN_encoding_genus, 
    suptitle = 'CNN models at Genus-level\nwith 3-, 5-mer and 7-mer encoded sequences')

**Comparing the BiLSTM model at Genus level with regular and mutation rate adjusted one-hot-encoded sequences**



In [ ]:
print(give_score(
    model = ['BiLSTM'], 
    taxon = ['Genus'], 
    encoding = ['regular one-hot', 'mutation rate adjusted one-hot'], 
    data = ['non-augmented']
    ))

plot_hist(
    histories_BiLSTM_encoding_genus, 
    suptitle = 'BiLSTM models at Genus-level\nwith regular and mutation rate adjusted one-hot-encoded sequences')

**Comparing the ConvBiLSTM model at Genus level with regular and mutation rate adjusted one-hot-encoded sequences**



In [ ]:
print(give_score(
    model = ['ConvBiLSTM'], 
    taxon = ['Genus'], 
    encoding = ['regular one-hot', 'mutation rate adjusted one-hot'], 
    data = ['non-augmented']
    ))

plot_hist(
    histories_ConvBiLSTM_encoding_genus, 
    suptitle = 'ConvBiLSTM models at Genus-level\nwith regular and mutation rate adjusted one-hot-encoded sequences')

**Comparing the Read2Pheno model at Genus level with regular and mutation rate adjusted one-hot-encoded sequences**

In [ ]:
print(give_score(
    model = ['R2P'], 
    taxon = ['Genus'], 
    encoding = ['regular one-hot', 'mutation rate adjusted one-hot'], 
    data = ['non-augmented']
    ))

plot_hist(
    histories_R2P_encoding_genus, 
    suptitle = 'Read2Pheno models at Genus-level\nwith regular and mutation rate adjusted one-hot-encoded sequences')

#### 2. Comparing the models' results at different taxon levels
**Comparing the CNN model at Family, Genus and Species level with 7-mer encoded sequences**



In [ ]:
print(give_score(
    model = ['CNN'], 
    taxon = ['Family', 'Genus', 'Species'], 
    encoding = ['7-mer'], 
    data = ['non-augmented']
    ))

plot_hist(
    histories_CNN_taxa, 
    suptitle = 'CNN models at Family, Genus and Species level\nwith 7-mer encoded sequences')

**Comparing the BiLSTM model at Family, Genus and Species level with mutation rate adjusted one-hot-encoded sequences**



In [ ]:
print(give_score(
    model = ['BiLSTM'], 
    taxon = ['Family', 'Genus', 'Species'], 
    encoding = ['mutation rate adjusted one-hot'], 
    data = ['non-augmented']
    ))

plot_hist(
    histories_BiLSTM_taxa, 
    suptitle = 'BiLSTM models at Family, Genus and Species level\nwith mutation rate adjusted one-hot-encoded sequences')

**Comparing the ConvBiLSTM model at Family, Genus and Species level with mutation rate adjusted one-hot-encoded sequences**



In [ ]:
print(give_score(
    model = ['ConvBiLSTM'], 
    taxon = ['Family', 'Genus', 'Species'], 
    encoding = ['mutation rate adjusted one-hot'], 
    data = ['non-augmented']
    ))

plot_hist(
    histories_ConvBiLSTM_taxa, 
    suptitle = 'ConvBiLSTM models at Family, Genus and Species level\nwith mutation rate adjusted one-hot-encoded sequences')

**Comparing the Read2Pheno model at Family, Genus and Species level with mutation rate adjusted one-hot-encoded sequences**



In [ ]:
print(give_score(
    model = ['R2P'], 
    taxon = ['Family', 'Genus', 'Species'], 
    encoding = ['mutation rate adjusted one-hot'], 
    data = ['non-augmented']
    ))

plot_hist(
    histories_R2P_taxa, 
    suptitle = 'Read2Pheno models at Family, Genus and Species level\nwith mutation rate adjusted one-hot-encoded sequences')

#### 3. Comparing the models' results with the regular, augmented and V-region selected data
**Comparing the CNN model at Genus level with 7-mer encoded sequences on the non-augmented, augmented and V-region selected data**



In [ ]:
print(give_score(
    model = ['CNN'], 
    taxon = ['Genus'], 
    encoding = ['7-mer'], 
    data = ['non-augmented', 'augmented', 'V-region selected']
    ))

plot_hist(
    histories_CNN_data, 
    suptitle = 'CNN models at Genus level with 7-mer encoded sequences\non non-augmented, augmented and V-region selected data')

**Comparing the BiLSTM model at Genus level with mutation rate adjusted one-hot-encoded sequences on the non-augmented, augmented and V-region selected data**



In [ ]:
print(give_score(
    model = ['BiLSTM'], 
    taxon = ['Genus'], 
    encoding = ['mutation rate adjusted one-hot'], 
    data = ['non-augmented', 'augmented', 'V-region selected']
    ))

plot_hist(
    histories_BiLSTM_data, 
    suptitle = 'BiLSTM models at Genus level with mutation rate adjusted one-hot-encoded sequences\non non-augmented, augmented and V-region selected data')

**Comparing the ConvBiLSTM model at Genus level with mutation rate adjusted one-hot-encoded sequences on the non-augmented, augmented and V-region selected data**



In [ ]:
print(give_score(
    model = ['ConvBiLSTM'], 
    taxon = ['Genus'], 
    encoding = ['mutation rate adjusted one-hot'], 
    data = ['non-augmented', 'augmented', 'V-region selected']
    ))

plot_hist(
    histories_ConvBiLSTM_data, 
    suptitle = 'ConvBiLSTM models at Genus level with mutation rate adjusted one-hot-encoded sequences\non non-augmented, augmented and V-region selected data')

**Comparing the Read2Pheno model at Genus level with mutation rate adjusted one-hot-encoded sequences on the non-augmented, augmented and V-region selected data**



In [ ]:
print(give_score(
    model = ['R2P'], 
    taxon = ['Genus'], 
    encoding = ['mutation rate adjusted one-hot'], 
    data = ['non-augmented', 'augmented', 'V-region selected']
    ))

plot_hist(
    histories_R2P_data, 
    suptitle = 'Read2Pheno models at Genus level with mutation rate adjusted one-hot-encoded sequences\non non-augmented, augmented and V-region selected data')

#### 4. Comparing the models with each other
**Comparing the RDP / CNN / BiLSTM / ConvBiLSTM / Read2Pheno models at species level on the non-augmented data**



In [ ]:
print(give_score(
    model = ['RDP', 'CNN', 'BiLSTM', 'ConvBiLSTM', 'R2P'], 
    taxon = ['Species'], 
    encoding = ['8-mer', '7-mer', 'mutation rate adjusted one-hot'], 
    data = ['non-augmented']
    ))

plot_hist(
    histories_models_spe, 
    suptitle = 'CNN / BiLSTM / ConvBiLSTM / Read2Pheno models at Species level\non the non-augmented data')

**Comparing the RDP / CNN / BiLSTM / ConvBiLSTM / Read2Pheno models at genus level on the non-augmented data**



In [ ]:
print(give_score(
    model = ['RDP', 'CNN', 'BiLSTM', 'ConvBiLSTM', 'R2P'], 
    taxon = ['Genus'], 
    encoding = ['8-mer', '7-mer', 'mutation rate adjusted one-hot'], 
    data = ['non-augmented']
    ))

plot_hist(
    histories_models_gen, 
    suptitle = 'CNN / BiLSTM / ConvBiLSTM / Read2Pheno models at Genus level\non the non-augmented data')

**Comparing the RDP / CNN / BiLSTM / ConvBiLSTM / Read2Pheno models at family level on the non-augmented data**



In [ ]:
print(give_score(
    model = ['RDP', 'CNN', 'BiLSTM', 'ConvBiLSTM', 'R2P'], 
    taxon = ['Family'], 
    encoding = ['8-mer', '7-mer', 'mutation rate adjusted one-hot'], 
    data = ['non-augmented']
    ))

plot_hist(
    histories_models_fam, 
    suptitle = 'CNN / BiLSTM / ConvBiLSTM / Read2Pheno models at Family level\non the non-augmented data')

**Comparing the CNN / BiLSTM / ConvBiLSTM / Read2Pheno models on the augmented data**



In [ ]:
print(give_score(
    model = ['RDP', 'CNN', 'BiLSTM', 'ConvBiLSTM', 'R2P'], 
    taxon = ['Genus'], 
    encoding = ['8-mer', '7-mer', 'mutation rate adjusted one-hot'], 
    data = ['augmented']
    ))

plot_hist(
    histories_models_a, 
    suptitle = 'CNN / BiLSTM / ConvBiLSTM / Read2Pheno models at Genus level\non the augmented data')

**Comparing the CNN / BiLSTM / ConvBiLSTM / Read2Pheno models on the V-region selected data**



In [ ]:
print(give_score(
    model = ['RDP', 'CNN', 'BiLSTM', 'ConvBiLSTM', 'R2P'], 
    taxon = ['Genus'], 
    encoding = ['8-mer', '7-mer', 'mutation rate adjusted one-hot'], 
    data = ['V-region selected']
    ))

plot_hist(
    histories_models_v, 
    suptitle = 'CNN / BiLSTM / ConvBiLSTM / Read2Pheno models at Genus level\non the V-region selected data')